In [1]:
%pip install -q langchain langchain-gigachat langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 2.4 MB/s eta 0:00:00


In [3]:
%pip install -q transformers sentence-transformers

In [4]:
import numpy as np
import torch
import torch.nn.functional as F
from torch import Tensor, inference_mode
from transformers import AutoTokenizer, AutoModel

In [5]:
tokenizer = AutoTokenizer.from_pretrained("deepvk/USER-bge-m3")
model = AutoModel.from_pretrained("deepvk/USER-bge-m3")
model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/697 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(46166, 1024, padding_idx=1)
    (position_embeddings): Embedding(8194, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-23): 24 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, eleme

In [6]:
with open("/content/result.txt", "r", encoding="utf-8") as file:
  text = file.read()

len(text)

85234

In [7]:
sentences = text.split("\n\n")

len(sentences)

349

In [8]:
embeddings_list = []

for s in sentences:
    encoded_input = tokenizer(
        s, padding=True, truncation=True, max_length=64, return_tensors="pt"
      )
    with torch.no_grad():
        model_output = model(**encoded_input)
    embedding = model_output.pooler_output
    embeddings_list.append((embedding)[0].numpy())

embeddings = np.asarray(embeddings_list)

In [9]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity

def determine_k(embeddings):
    k_min = 10
    clusters = [x for x in range(2, k_min * 11)]
    metrics = []
    for i in clusters:
        metrics.append((KMeans(n_clusters=i).fit(embeddings)).inertia_)
    k = elbow(k_min, clusters, metrics)
    return k

def elbow(k_min, clusters, metrics):
    score = []

    for i in range(k_min, clusters[-3]):
        y1 = np.array(metrics)[:i + 1]
        y2 = np.array(metrics)[i:]

        df1 = pd.DataFrame({'x': clusters[:i + 1], 'y': y1})
        df2 = pd.DataFrame({'x': clusters[i:], 'y': y2})

        reg1 = LinearRegression().fit(np.asarray(df1.x).reshape(-1, 1), df1.y)
        reg2 = LinearRegression().fit(np.asarray(df2.x).reshape(-1, 1), df2.y)

        y1_pred = reg1.predict(np.asarray(df1.x).reshape(-1, 1))
        y2_pred = reg2.predict(np.asarray(df2.x).reshape(-1, 1))

        score.append(mean_squared_error(y1, y1_pred) + mean_squared_error(y2, y2_pred))

    return np.argmin(score) + k_min

k = determine_k(embeddings)

In [10]:
k_opt = k

In [11]:
from sklearn.metrics.pairwise import euclidean_distances

kmeans = KMeans(n_clusters = k_opt, random_state = 42).fit(embeddings)
kmeans_labels = kmeans.labels_

data = pd.DataFrame()
data['text'] = sentences
data['label'] = kmeans_labels
data['embedding'] = list(embeddings)

kmeans_centers = kmeans.cluster_centers_
top_texts_list = []
for i in range (0, k_opt):
    cluster = data[data['label'] == i]
    embeddings = list(cluster['embedding'])
    texts = list(cluster['text'])
    distances = [euclidean_distances(kmeans_centers[0].reshape(1, -1), e.reshape(1, -1))[0][0] for e in embeddings]
    scores = list(zip(texts, distances))
    top_3 = sorted(scores, key=lambda x: x[1])[:3]
    top_texts = list(zip(*top_3))[0]
    top_texts_list.append(top_texts)

In [12]:
data.head(5)

,text,label,embedding
0,"speaker_id: 0, text: Я вам говорю именно про т...",13,"[-0.7976565, 0.5811596, 0.07980482, 0.02231400..."
1,"speaker_id: -1, text: Я вам говорю именно про ...",19,"[-0.638559, 0.27276018, 0.42071015, 0.48895913..."
2,"speaker_id: 2, text: Давайте сюда строчку, ещё...",42,"[-0.5856101, 0.73511547, -0.37175333, 0.007708..."
3,"speaker_id: 1, text: Нет, на лягу. Да какая ра...",16,"[-0.21750924, 0.45731756, -0.087864034, 0.6429..."
4,"speaker_id: 2, text: Я то скопировала не сюда,...",14,"[-0.60595506, 0.2754217, -0.24061905, -0.04133..."


In [13]:
# Получаем объединенные тексты для каждого кластера
combined_texts_per_cluster = {}

for i in range(k_opt):
    cluster_texts = data[data['label'] == i]['text'].tolist()
    combined_text = ' '.join(cluster_texts)
    combined_texts_per_cluster[i] = combined_text

# Выводим результаты
for cluster_id, combined_text in combined_texts_per_cluster.items():
    print(f"Кластер {cluster_id}:")
    print(f"Количество текстов: {len(data[data['label'] == cluster_id])}")
    print(f"Объединенный текст (первые 200 символов): {combined_text[:200]}...")
    print("-" * 80)

Кластер 0:
Количество текстов: 5
Объединенный текст (первые 200 символов): speaker_id: 2, text: Если это крепление, и оно заняло там, не знаю, 5 или 6 суток, а мы бурение закончили в начале месяца. Да, конечно, мы к концу месяца примем и бурение, и крепление не, и все сервис...
--------------------------------------------------------------------------------
Кластер 1:
Количество текстов: 11
Объединенный текст (первые 200 символов): speaker_id: 2, text: А почему у тебя вмр вручную?, emotion: neutral speaker_id: 2, text: Это из календарика или это и там?, emotion: neutral speaker_id: 2, text: Как бы свои то у нас отдельно., emotio...
--------------------------------------------------------------------------------
Кластер 2:
Количество текстов: 3
Объединенный текст (первые 200 символов): speaker_id: 1, text: Не, это по, это может быть тот же договор. На самом деле газпром брине тоже есть вручную статьи затрат., emotion: neutral speaker_id: 1, text: Нет, в договоре я бы их все равно ра...


In [15]:
combined_texts_per_cluster[12]

'speaker_id: -1, text: Ну, мы прём в эту простую статью затрат, которая будет на все этапы, она у тебя в любом случае будет идти отдельно на все, ну, на секундочку а как она, она все идёт на tk, я же вам рассказывал да, да, я понимаю а на вот это вот, тк. Она отдельно, ежемесячно, ежедневно. Не не не, не, не, на ежедневно. Ну, мы это поняли ксг, откройте, пожалуйста., emotion: neutral speaker_id: 2, text: В любом случае будет идти отдельно на все. Ты её не привяжешь проблем, Виктор Викторович, она итк отдельно рассказывал. А у нас вот эта вот итк, она отдельно, ежемесячно. А, не, не, не, не, ну мы это поняли. Ксг, откройте, пожалуйста., emotion: neutral speaker_id: 2, text: Она в ксг отдельной строчечкой её вообще нет, значит, вы её просто приплюсуете к какому-то этапу и да нет, ну не будет она у тебя, знаешь, как ляжет, как в договоре, на какие этапы распространяется и tk., emotion: neutral speaker_id: -1, text: Она в ксг отдельной строчкой её вообще нету, её вообще нет, она в проекте

In [18]:
prompt = """Ты - профессиональный ассистент для суммаризации аудио транскрипций. Твоя задача - создать четкую, структурированную и информативную summary аудиозаписи.

**ИСХОДНЫЙ ТЕКСТ:**
{text}

**ИНСТРУКЦИИ ДЛЯ SUMMARY:**

1. **ТИП SUMMARY:** Создай конспект в формате "ключевые идеи + выводы"
2. **СТРУКТУРА:**
   - **Основная тема:** 1-2 предложения
   - **Ключевые моменты:** 3-5 самых важных идей
   - **Конкретные примеры/факты:** 2-3 значимых детали
   - **Выводы/рекомендации:** что следует запомнить или сделать
   - **Общий тон/настроение:** нейтральный, информативный, критический и т.д.
   - Сохраняй ключевые имена, абривеатуры и названия

3. **ОСОБЕННОСТИ АУДИО ТРАНСКРИПЦИЙ:**
   - Игнорируй слова-паразиты, повторы, паузы ("э-э-э", "короче", "типа")
   - Сохрани важные эмоциональные акценты, если они есть
   - Учитывай, что это устная речь - могут быть незаконченные мысли

4. **СТИЛЬ:** Профессионально, но доступно. Используй маркированные списки для наглядности.

**РЕЗУЛЬТАТ ДОЛЖЕН БЫТЬ ПОЛЕЗЕН ДЛЯ ТОГО, КТО НЕ СЛУШАЛ АУДИОЗАПИСЬ.**
"""

In [17]:
%pip install -q langchain langchain-gigachat

In [29]:
from langchain_gigachat import GigaChat
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter

from google.colab import userdata

In [23]:
llm = GigaChat(
    credentials=userdata.get("GIGACHAT_APIKEY"),
    scope=userdata.get("GIGACHAT_SCOPE"),
    model="gigachat:latest",
    profanity_check=False,
    verify_ssl_certs=False,
)

In [30]:
from tqdm import tqdm

chain = ChatPromptTemplate.from_template(prompt) | llm | StrOutputParser()

summaries = []
for cluter_id, text in tqdm(combined_texts_per_cluster.items()):
  summary = chain.invoke({"text": text})
  summaries.append(summary)

100%|██████████| 43/43 [01:22<00:00,  1.91s/it]


In [31]:
summary = "\n\n".join(summaries)

len(summary)

49386

In [32]:
MEETING_PROTOCOL_PROMPT = """Задача: На основе транскрибации (текстовой расшифровки) совещания составить официальный протокол в структурированном формате.

## Шаг 1. Вводные данные
 * Название компании: [вписать]

 * Вид документа: [например, "Протокол производственного совещания"]

 * Дата проведения: [дд.мм.гггг]

 * Место проведения: [если важно]

## Шаг 2. Участники
 * Председатель: [ФИО, способ участия (лично/онлайн)]

 * Секретарь: [ФИО]

 * Присутствовали:

 * * [ФИО] – лично/онлайн

 * * [ФИО] – лично/онлайн
(и т. д.)

## Шаг 3. Повестка дня
Перечислить основные вопросы, которые обсуждались:

1. [Тема 1]

2. [Тема 2]

3. [Прочие вопросы]

## Шаг 4. Обсуждение (СЛУШАЛИ)
Кратко изложить ключевые моменты выступлений:

 * [ФИО или "Всех присутствующих"]: [основные тезисы, мнения, предложения]

## Шаг 5. Решения (ПОСТАНОВИЛИ)
Оформить в виде таблицы (если нужно) или списка:

| № п/п | Содержание поручения | Ответственный исполнитель (ФИО) | Срок выполнения |
| 1.1 | [Суть задачи] |	[ФИО] |	[дд.мм.гггг] |
| 1.2 |	[Суть задачи] |	[ФИО] |	[дд.мм.гггг] |
Или списком:

[Задача] – [Исполнитель], срок до [дата].

## Шаг 6. Подписи
Председатель: __________ / [ФИО]

Секретарь: __________ / [ФИО]

Дополнительные указания:
Сохранять официально-деловой стиль.

Избегать лишних деталей, оставлять только ключевые решения.

Если в обсуждении были разногласия – указать альтернативные мнения.

Даты и сроки проверять на актуальность.

Пример заполнения:
"1.1 Восстановить документы по проекту 'Алькор' – Кравцова Л.М., срок до 10.06.2025"

Формат вывода: Готовый протокол в формате, аналогичном образцу.


Транскрибация совещания:
{summary}
"""

In [33]:
chain = ChatPromptTemplate.from_template(MEETING_PROTOCOL_PROMPT) | llm | StrOutputParser()

meeting_protocol = chain.invoke({"summary": summary})

with open(
    "GigaChat_lite_semantic_map_reduce_Протокол_совещания.txt", "w", encoding="utf-8"
  ) as file:
  file.write(meeting_protocol)